In [8]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time
import math

base = BaseOverlay("base.bit")


    # 定义分类器
cascade_path = 'cascade.xml'
cascade_car_path = 'cars.xml'
car_cascade = cv2.CascadeClassifier(cascade_car_path)
cascade = cv2.CascadeClassifier(cascade_path)


# monitor configuration: 640*480 @ 24Hz
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# monitor (output) frame buffer size
frame_out_w = 640
frame_out_h = 480

scale=400/480 #比例系数

PLATE_WIDTH=0.44


In [10]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640);
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480);

print("Capture device is open: " + str(cap.isOpened()))

Capture device is open: True


In [13]:
time_start=time.time()
time_last=0
i=1
x_last=0
y_last=0
while(cap.isOpened()):
    time_start=time.time()
    ret, frame = cap.read()
    #print("after read time is:"+str(time.time()-time_start))
    if (ret):      
        time_last=time_start
        time_start=time.time()
        
        image=cv2.resize(frame,(int(640*scale),400))
        outframe = hdmi_out.newframe()
        #print("after new outframe time is:"+str(time.time()-time_start))
        img_gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        #print("after cvtgray time is:"+str(time.time()-time_start))
        car_plates = cascade.detectMultiScale(img_gray, 1.1, 2, minSize=(36, 9), maxSize=(36 * 40, 9 * 40))
        #print("after mul time is:"+str(time.time()-time_start))
        for car_plate in car_plates:
            x, y, w, h = car_plate
            #plate = image[y: y + h, x: x + w]
            #if(isPlate(plate)==False):#根据颜色判断是否是正确的车牌区域
            #    continue
            #if(access_pixels(plate)==False):#根据颜色判断是否是正确的车牌区域
            #    continue
            #print('accessPlate time:',time.time()-time_start,'s')
            #print('time cost:',time.time()-time_start,'s')
            #标出粗定位的车牌
            cv2.rectangle(image, (x - 10, y - 10), (x + w + 10, y + h + 10), (255, 255, 255), 1)
            plateScale=(w/PLATE_WIDTH)
            v=(math.sqrt((x-x_last)*(x-x_last)+(y-y_last)*(y-y_last))/(time_start-time_last)/plateScale)/i
            x_last=x
            y_last=y
            i=0
            
            cv2.putText(image,"Vehicle Velocity:"+str(v)+" m/s",(10,image.shape[0]-10),cv2.FONT_HERSHEY_PLAIN, 2.0, (0, 0, 255), 2)
        
        image=cv2.resize(image,(640,480))
        outframe[0:640,0:480,:] = image[0:640,0:480,:]
        #print("after copy time is:"+str(time.time()-time_start))
        
        hdmi_out.writeframe(outframe)
        #print("after write time is:"+str(time.time()-time_start))
        i+=1
        
       
    else:
        raise RuntimeError("Failed to read from camera.")

KeyboardInterrupt: 